<a href="https://colab.research.google.com/github/PrashanthiVelpula/project/blob/main/MinorProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipython-autotime
%load_ext autotime

In [ ]:
import os
import numpy as np

import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
 
target = []
images=[]
flat_data = []

datadir ='/content/drive/MyDrive/Images'

categories = ['banana','grapes','mango']

for i in categories:
  class_num=categories.index(i)
  path = os.path.join(datadir,i)
  for img in os.listdir(path):
      img_array = imread(os.path.join(path,img))
      img_resized = resize(img_array,(150,150,3)) 
      flat_data.append(img_resized.flatten())
      images.append(img_resized)
      target.append(class_num)
flat_data = np.array(flat_data)
target = np.array(target)
images=np.array(images)
      

In [ ]:
# Splitting the data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(flat_data,target,test_size=0.25,random_state=100)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm
param_grid = [{'C':[1,10,100,1000],'kernel':['linear']},
              {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']},]
svc=svm.SVC(probability=True)
clf=GridSearchCV(svc,param_grid)
clf.fit(x_train,y_train)

In [ ]:
y_pred=clf.predict(x_test)
y_pred

In [ ]:
y_test

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [ ]:
accuracy_score(y_pred,y_test)

In [ ]:
confusion_matrix(y_pred,y_test)

In [ ]:
import pickle
pickle.dump(clf,open('img_model.p','wb'))

In [ ]:
model=pickle.load(open('img_model.p','rb'))

In [ ]:
flat_data=[]
url = input('Enter your url')
img = imread(url)
img_resized=resize(img,(150,150,3))
flat_data.append(img_resized.flatten())
flat_data=np.array(flat_data)
print(img.shape)
y_out=model.predict(flat_data)
y_out=categories[y_out[0]]
print(f'PREDICTED OUTPUT: {y_out}')

In [ ]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

In [ ]:
%%writefile App.py
import streamlit as st
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import pickle
from PIL import Image
st.title('Image Classification  using Machine learning')
st.text('Upload the image ')
model=pickle.load(open('img_model.p','rb'))
uploaded_file=st.file_uploader("Choose an image....",type='jpg')
if uploaded_file is not None:
  img=Image.open(uploaded_file)
  st.image(img,caption='Uploaded Image')
  if st.button('PREDICT'):
    categories = ['banana','grapes','mango']
    st.write('Result.....')
    flat_data=[]
    img =np.array(img)
    img_resized=resize(img,(150,150,3))
    flat_data.append(img_resized.flatten())
    flat_data=np.array(flat_data)
    y_out=model.predict(flat_data)
    y_out=categories[y_out[0]]
    st.title(f'PREDICTED OUTPUT: {y_out}')

In [ ]:
!nohup streamlit run App.py &
url=ngrok.connect(port='8501')
url
